# Read .wav file and make 3-D animated spectrogram

This script uses the GR framework to make an animated 3d spectrogram from a sound file.  More details on the GR framework are at https://gr-framework.org/index.html
<br>
One on-going problem is how to view the output.  The script works great when run as python script, but within the jupyter notebook all I can do is save a quicktime movie.  A question has been posted to StackOverflow.

In [1]:
# import standard modules; note we need pyaudio and struct like before, added this time is
# gr and gr3
import os, wave, pyaudio
import numpy as np
import gr
import gr3

Here's an attempt to include the output within the notebook.  This gives an error "No video with supported format and MIME type found".  What it does do is save a quicktime movie (gks.mov) file (no sound though...)
Conversely, I can can't figure out how to get the output into a separate window.

In [2]:
gr.inline("mp4")

In [3]:
# set sampling frequency (usually 44100)
RATE = 44100
# set samples per plot
CHUNK = 2048

dt = CHUNK/RATE
df = RATE/CHUNK/4
spectrum = np.zeros((256, 64), dtype=float)

Read a wav file as input

In [4]:
wf = wave.open(('/Users/jimp/python_examples/sound/sound_files/T08-violin.wav'), 'rb')

Setup pyaudio to read from file (wf)

In [5]:
# Before we specified this as:
#   format = pyaudio.paInt16
#   channels = 1
#   rate = RATE
#   output to True

pa = pyaudio.PyAudio()
stream = pa.open(format=pa.get_format_from_width(wf.getsampwidth()),
                 channels=wf.getnchannels(), rate=wf.getframerate(), output=True)

In [6]:
# set initial time (arbitrary?  but this will start at t, if negative get white space
#   in the graph until t reaches zero). I think since we set the spectrum to be 256 x 64, this
#   gives us each column
t = -63

We want to display a moving spectrogram (aka waterfall plot).  One axis will be frequency, and the other time.  The time part will scroll.  The spectra values will be colorshaded as well as height along the z-axis.

The gr module documentation is at https://gr-framework.org/_modules/gr.html

In [7]:
# read from the wav file a "CHUNK's worth" of data
data = wf.readframes(CHUNK)

# now loop over all "CHUNKS"
while data != '' and len(data) == CHUNK * wf.getsampwidth():
    stream.write(data)
    amplitudes = np.fromstring(data, dtype=np.short)
    S2 = int(CHUNK/2)
#    power = abs(np.fft.fft(amplitudes / 32768.0))[:CHUNK/2]
    power = abs(np.fft.fft(amplitudes / 32768.0))[:S2]

    gr.clearws()
# here we compute the spectrum, then "roll" every point (this makes the spectrogram appear
# to scroll; a +1 would scroll right one each time-step, while a -1 would scroll left)    
    spectrum[:, 63] = power[:256]
    spectrum = np.roll(spectrum, -1)
    
# specify the colormap; don't know where these are defined
    gr.setcolormap(-113)
    
# set the (xmin,xmax,ymin,ymax) of the viewport; these are defined 0 to 1, so a value
#   of (0,1,0,1) means the graphic will go right to the edge of the page (no white space)
    gr.setviewport(0.05, 0.95, 0.1, 1)
    
# set the graph range (xmin,xmax,ymin,ymax)
#   here we will have a graph with a constant y-axis (frequency) and an x-axis that
#   is fixed width (64 time-steps) but changes with each delta-t
    gr.setwindow(t * dt, (t + 63) * dt, 0, df)
    
# set zmin, zmax, rotation of x-axis, and z-axis tilt
#    x-axis rotation 0 means horizontal along x
#    x-axis rotation 45 means x runs SE (origin) to NW (max x)
#    z-axis tilt 0 means a view from the "table edge"
#    z-axis tilt 90 means a view from the "table top"for 3-D plot set zmin, zmax, rotation of x-axis, tilt
    gr.setspace(0, 256, 30, 80)
    
# draw surface ( x-coord, y-coord, z-coord, type ) where
#    type = 0: lines
#           1: mesh
#           2: filled mesh
#           3: z-shaded mesh
#           4: colored mesh
#           5: cell array
#           6: shaded mesh
    gr3.surface((t + np.arange(64)) * dt, np.linspace(0, df, 256), spectrum, 4)

# set the axis type (can be FLIP and/or LOG for each axis, e.g., OPTION_Y_LOG, OPTION_FLIP_Y)
#    gr.setscale()

# set axes/ticks (x-tick interval, y-tick interval, z-tick interval, 
#                 x-origin, y-origin, z-origin, number of minor tick
#                 marks between major tick marks (x, y, z), tick size
#                 negative for outward facing)
    gr.axes3d(0.2, 0.2, 0, (t + 63) * dt, 0, 0, 5, 5, 0, 0.01)
    
# label graph
    gr.titles3d('t [s]', 'f [kHz]', '')
    
# update graphic    
    gr.updatews()

    data = wf.readframes(CHUNK)
    t += 1

In [8]:
gr.show()